### Model
supplier:
$\begin{aligned} \dot{s}^{S}(\tau) &=o_{r}^{S}\left(\tau-\tau_{2}\right)-o_{r}^{R}\left(\tau-\tau_{1}\right)-b^{S}(\tau) / t_{b} \\ \dot{o}_{u}^{S}(\tau) &\left.=o_{r}^{S}(\tau)-o_{r}^{S}\left(\tau-\tau_{2}\right)\right) \\ \dot{b}^{S}(\tau) &=o_{r}^{R}(\tau)-o_{r}^{R}\left(\tau-\tau_{1}\right)-b^{S}(\tau) / t_{b} \end{aligned}$

retailer: 
$\begin{aligned} \dot{s}^{R}(\tau)&=o_{r}^{R}\left(\tau-\tau_{1}-\tau_{2}\right)+b^{S}\left(\tau-\tau_{2}\right) / t_{b} -d_{r}\left(\tau-\tau_{1}\right)-b^{R}(\tau) / t_{b}\\
\dot{o}_{u}^{R}(\tau)&=o_{r}^{R}(\tau)-o_{r}^{R}\left(\tau-\tau_{1}-\tau_{2}\right)-b^{S}\left(\tau-\tau_{2}\right) / t_{b}\\
\dot{b}^{R}(\tau)&=d_{r}^{R}(\tau)-d_{r}^{R}\left(\tau-\tau_{1}\right)-b^{R}(\tau) / t_{b} \end{aligned}$

$\tau_1 = 1$, $\tau_2 = 2$

Transfer into Laplace domain and approximate with 3rd order Pade approximation with

$u_1 = o_r^S$, $u_2 = o_r^R$, $u_3 = d_r^R$

$x_1 = s^S$, $x_2 = o^S_u$, $x_3 = b^S$, $x_4 = z_{11}$, $x_5 = \dot{z}_{11}$, $x_6 = z^{(2)}_{11}$, $x_7 = z_{12}$, $x_8 = \dot{z}_{12}$, $x_9 = z^{2}_{12}$, $x_{10} = s^{R}$, $x_{11} = o_u^R$, $x_{12} = b^R$, $x_{13} = z_{13}$, $x_{14} = \dot{z}_{13}$, $x_{15} = z^{(2)}_{13}$, $x_{16} = z_{14}$, $x_{17} = \dot{z}_{14}$, $x_{18} = z^{(2)}_{14}$, $x_{19} = z_{15}$, $x_{20} = \dot{z}_{15}$, $x_{21} = z^{(2)}_{15}$


$z_{11} = \frac{1}{s^{3} + 12 s^2 + 60 s +120}u_1$, $z_{12} = \frac{1}{s^3 + 6s^2 + 15s +15}u_2$, $z_{13} = \frac{1}{s^3 + 4s^2 + 6.667s + 4.444}u_1$, $z_{14} = \frac{1}{s^{3} + 12 s^2 + 60 s +120}u_2$, $z_{15} = \frac{1}{s^3 + 6s^2 + 15s +15}u_3$

#### State space
$\begin{aligned}\dot{x}_1 &=  -u_1 + 24x_6 + 240x_4 + u_2 - 12x_9 - 30x_7 -x_3/t_b \\\dot{x}_2 &=  2u_1 - 24x_6 - 240x_4\\\dot{x}_3 &=  2u_2 - 12x_9  - 30x_7 -x_3/t_b \\ \dot{x}_{10} &= -u_2 + 8x_{15} + 8.888x_{13} + (-x_3 + 24x_{18} +240x_{16})/t_b  +u_3 -12x_{21} - 30x_{19} - x_{12}/t_b\\ \dot{x}_{11} &= 2u_2 -8x_{15} - 8.888x_{13} -(-x_3 + 24x_{18} + 240x_{16})/t_b\\ \dot{x}_{12} &= 2u_3 -12x_{21} -30x_{19} - x_{12}/t_b        \end{aligned}$

In [2]:
import numpy as np
import casadi as ca

In [10]:
Nx = 21
Nu = 3
x_SX = ca.SX.sym("x",Nx)
u_SX = ca.SX.sym("u",Nu)
tb = 1

In [7]:
A = ca.DM([
    [0, 0, -1/tb, 240, 0, 24, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 240, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1/tb, 0, 0, 0, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, -120, -60, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -15, -15, -6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, -1/tb, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 8.888, 0, 8, 240/tb, 0, 24/tb, -30, 0, -12],    
    [0, 0, 1/tb, 0, 0, 0, 0, 0, 0, 0, 0, 0, -8.888, 0, -8, -240/tb, 0, -24/tb, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 0, 0, 0, 0, 0, 0, -30, 0, -12],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4.444, -6.667, -4, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -120, -60, -12, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -15, -6]
])

In [9]:
B = ca.DM([
    [-1,1,0],
    [2,0,0],
    [0,2,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,-1,1],
    [0,2,0],
    [0,0,2],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0]
])

In [11]:
x_dot = A@x_SX + B@ u_SX